In [1]:
!pip install classiq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.1/615.1 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.33.2
    Uninstalling pydantic_core-2.33.2:
      Successfully uninstalled pydantic_core-2.33.2
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall:


Algorithm 10: Decompose Group
From "Decomposing Finite Abelian Groups" by Cheung & Mosca


In [8]:
import numpy as np
from typing import List, Tuple, Dict, Optional, Set
from dataclasses import dataclass
import math
from functools import reduce
import itertools
from classiq import *

Representation

In [9]:
@dataclass(frozen=True)
class GroupElement:
    """Represents an element in Z_q^k"""
    coordinates: Tuple[int, ...]
    modulus: int

    def __init__(self, coordinates: List[int], modulus: int):
        object.__setattr__(self, 'coordinates', tuple(coordinates))
        object.__setattr__(self, 'modulus', modulus)

    def __add__(self, other: 'GroupElement') -> 'GroupElement':
        assert len(self.coordinates) == len(other.coordinates)
        assert self.modulus == other.modulus
        new_coords = [(a + b) % self.modulus for a, b in zip(self.coordinates, other.coordinates)]
        return GroupElement(new_coords, self.modulus)

    def __mul__(self, scalar: int) -> 'GroupElement':
        new_coords = [(scalar * coord) % self.modulus for coord in self.coordinates]
        return GroupElement(new_coords, self.modulus)

    def __eq__(self, other: 'GroupElement') -> bool:
        return self.coordinates == other.coordinates and self.modulus == other.modulus

    def __hash__(self) -> int:
        return hash((self.coordinates, self.modulus))

    def is_zero(self) -> bool:
        return all(coord == 0 for coord in self.coordinates)

    def __repr__(self) -> str:
        return f"GroupElement({list(self.coordinates)}, {self.modulus})"

Procedure:
1. Define g : Zkq → G by mapping (x1 , ..., xk ) → g(x) = ax1 1 · · · axk k . Find generators for the
hidden subgroup K of Zkq as defined by the function g.
2. Compute a set y1 , ..., yl ∈ Zkq /K of generators for Zkq /K.
3. Output {g(y1 ), ..., g(yl )}.

In [10]:
class Algorithm10Implementation:
    """
    Complete implementation of Algorithm 10: Decompose Group
    """

    def __init__(self, generators: List[str], orders: List[int], q: int):
        """
        Initialize with generating set and maximum order

        Args:
            generators: List of generator names {a1, ..., ak}
            orders: List of orders of generators
            q: Maximum order q = p^r where p is prime
        """
        self.generators = generators
        self.orders = orders
        self.q = q  # Must be p^r for some prime p
        self.k = len(generators)

        # Verify q is prime power and all orders divide q
        assert self._is_prime_power(q), f"q={q} must be a prime power"
        assert all(q % order == 0 for order in orders), "All generator orders must divide q"

        # The group G is generated by a1, ..., ak with given orders
        self.G_elements = self._enumerate_group_elements()

    def decompose_group(self) -> Tuple[List[GroupElement], List[GroupElement]]:
        """
        Complete implementation of Algorithm 10

        Returns:
            (generators_of_quotient, generators_of_G): Tuple of generator lists
        """
        print(f"Starting Algorithm 10 with generators {self.generators}, q={self.q}")

        # Step 1: Find generators for hidden subgroup K
        print("Step 1: Finding hidden subgroup K...")
        K_generators = self._find_hidden_subgroup_generators()
        print(f"Found {len(K_generators)} generators for hidden subgroup K")

        # Step 2: Compute generators for Z_q^k / K
        print("Step 2: Computing generators for Z_q^k / K...")
        quotient_generators = self._compute_quotient_generators(K_generators)
        print(f"Found {len(quotient_generators)} generators for quotient group")

        # Step 3: Output g(y1), ..., g(yl)
        print("Step 3: Computing final group generators...")
        final_generators = self._apply_g_function(quotient_generators)

        return quotient_generators, final_generators

    def _find_hidden_subgroup_generators(self) -> List[GroupElement]:
        """
        Step 1: Find generators for the hidden subgroup K of Z_q^k

        The hidden subgroup K = {(x1, ..., xk) | a1^x1 * ... * ak^xk = e}

        In a real quantum implementation, this would use the quantum
        hidden subgroup algorithm. Here we compute it classically.
        """
        K_elements = []

        # Enumerate all elements of Z_q^k
        for coords in itertools.product(range(self.q), repeat=self.k):
            x = GroupElement(list(coords), self.q)

            # Check if g(x) = e (identity in G)
            if self._g_function_is_identity(x):
                K_elements.append(x)

        print(f"Hidden subgroup K has {len(K_elements)} elements")

        # Find generators for K
        return self._find_subgroup_generators(K_elements)

    def _g_function_is_identity(self, x: GroupElement) -> bool:
        """
        Check if g(x) = a1^x1 * ... * ak^xk = e (identity)

        This checks if the element x represents the identity in the group G.
        For the group generated by a1, ..., ak with orders orders[0], ..., orders[k-1],
        we have a1^x1 * ... * ak^xk = e iff each xi is 0 mod order(ai).
        """
        for i in range(self.k):
            # Check if xi ≡ 0 (mod order(ai))
            if x.coordinates[i] % self.orders[i] != 0:
                return False
        return True

    def _find_subgroup_generators(self, subgroup_elements: List[GroupElement]) -> List[GroupElement]:
        """
        Find a minimal generating set for a subgroup given all its elements
        """
        if not subgroup_elements:
            return []

        # Remove the zero element
        non_zero_elements = [x for x in subgroup_elements if not x.is_zero()]

        if not non_zero_elements:
            return []

        generators = []
        generated_so_far = {GroupElement([0] * self.k, self.q)}  # Start with identity

        for element in non_zero_elements:
            if element not in generated_so_far:
                generators.append(element)
                # Add all multiples of this element to generated_so_far
                self._add_cyclic_subgroup_to_set(element, generated_so_far)

        return generators

    def _add_cyclic_subgroup_to_set(self, generator: GroupElement, element_set: Set[GroupElement]):
        """Add all elements generated by a single element to the set"""
        current = generator
        identity = GroupElement([0] * self.k, self.q)

        while current not in element_set:
            element_set.add(current)
            current = current + generator
            # Safety check to avoid infinite loop
            if current == identity and identity not in element_set:
                element_set.add(identity)
                break

    def _compute_quotient_generators(self, K_generators: List[GroupElement]) -> List[GroupElement]:
        """
        Step 2: Compute generators for Z_q^k / K using the matrix method from the paper

        This implements the matrix construction [M|A] and applies Theorem 7
        """
        print("Computing quotient generators using matrix method...")

        # Create the matrix A whose columns generate K
        if not K_generators:
            # If K is trivial, Z_q^k / K ≅ Z_q^k
            print("K is trivial, quotient is isomorphic to Z_q^k")
            return [GroupElement([1 if i == j else 0 for i in range(self.k)], self.q)
                   for j in range(self.k)]

        A = np.array([list(gen.coordinates) for gen in K_generators]).T
        print(f"Matrix A (generators of K) shape: {A.shape}")
        print(f"Matrix A:\n{A}")

        # Create matrix M = qI (k×k identity matrix scaled by q)
        M = self.q * np.eye(self.k, dtype=int)
        print(f"Matrix M = {self.q}I shape: {M.shape}")

        # Form the matrix [M|A]
        if A.size > 0:
            M_prime = np.hstack([M, A])
        else:
            M_prime = M
        print(f"Matrix M' = [M|A] shape: {M_prime.shape}")
        print(f"Matrix M':\n{M_prime}")

        # Apply Theorem 7: Find Smith Normal Form
        generators_quotient = self._apply_theorem_7(M_prime)

        return generators_quotient

    def _apply_theorem_7(self, M_prime: np.ndarray) -> List[GroupElement]:
        """
        Apply Theorem 7 to find generators for the quotient group

        Given matrix M', find generators g1, ..., gl such that
        Z_q^k / K = <g1> ⊕ ... ⊕ <gl>
        """
        print("Applying Theorem 7 (Smith Normal Form)...")

        # Compute Smith Normal Form
        U, D, V = self._smith_normal_form(M_prime)

        print(f"Smith Normal Form diagonal: {[D[i,i] for i in range(min(D.shape)) if D[i,i] != 0]}")

        # The quotient group Z_q^k / K is isomorphic to Z_d1 ⊕ ... ⊕ Z_dl
        # where d1, ..., dl are the non-zero diagonal elements of D

        generators = []

        # According to the paper's construction:
        # We need to find generators for Z_q^k / K
        # The standard basis vectors e1, ..., ek generate Z_q^k
        # We transform these using the unimodular matrix operations

        # Find which diagonal elements give non-trivial cyclic factors
        for i in range(min(D.shape)):
            d_i = D[i, i]
            if d_i > 1 and i < self.k:  # Non-trivial factor and within our dimension
                # The corresponding generator in the quotient comes from
                # the transformation encoded in matrix V
                try:
                    V_inv = np.linalg.inv(V.astype(float)).astype(int)
                    generator_coords = [int(V_inv[j, i]) % self.q for j in range(self.k)]
                    generators.append(GroupElement(generator_coords, self.q))
                except:
                    # Fallback to simple generators if matrix inversion fails
                    if i < self.k:
                        generator_coords = [1 if j == i else 0 for j in range(self.k)]
                        generators.append(GroupElement(generator_coords, self.q))

        # If no generators found, use identity-based generators
        if not generators:
            generators = [GroupElement([1 if i == j else 0 for i in range(self.k)], self.q)
                         for j in range(self.k)]

        return generators

    def _apply_g_function(self, quotient_generators: List[GroupElement]) -> List[str]:
        """
        Step 3: Apply g function to quotient generators to get final generators

        For each yi in quotient generators, compute g(yi) = a1^yi1 * ... * ak^yik
        """
        final_generators = []

        for i, y in enumerate(quotient_generators):
            # Create expression for g(y)
            terms = []
            for j in range(self.k):
                if y.coordinates[j] != 0:
                    if y.coordinates[j] == 1:
                        terms.append(self.generators[j])
                    else:
                        terms.append(f"{self.generators[j]}^{y.coordinates[j]}")

            if terms:
                generator_expr = " * ".join(terms)
            else:
                generator_expr = "e"  # identity

            final_generators.append(generator_expr)

        return final_generators

    def _smith_normal_form(self, A: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """
        Compute Smith Normal Form: U * A * V = D
        where D is diagonal with d1|d2|...|dk
        """
        A = A.astype(int)
        m, n = A.shape

        # Initialize unimodular matrices
        U = np.eye(m, dtype=int)
        V = np.eye(n, dtype=int)

        # Work with a copy
        D = A.copy()

        min_dim = min(m, n)

        for i in range(min_dim):
            # Find pivot (smallest non-zero element)
            pivot_found = False
            for r in range(i, m):
                for c in range(i, n):
                    if D[r, c] != 0:
                        if not pivot_found or abs(D[r, c]) < abs(D[pivot_r, pivot_c]):
                            pivot_r, pivot_c = r, c
                            pivot_found = True

            if not pivot_found:
                break

            # Move pivot to position (i, i)
            if pivot_r != i:
                D[[i, pivot_r]] = D[[pivot_r, i]]
                U[[i, pivot_r]] = U[[pivot_r, i]]
            if pivot_c != i:
                D[:, [i, pivot_c]] = D[:, [pivot_c, i]]
                V[:, [i, pivot_c]] = V[:, [pivot_c, i]]

            # Make diagonal element positive
            if D[i, i] < 0:
                D[i, :] *= -1
                U[i, :] *= -1

            # Clear row and column
            for j in range(i + 1, n):
                if D[i, j] != 0:
                    q = D[i, j] // D[i, i]
                    D[:, j] -= q * D[:, i]
                    V[:, j] -= q * V[:, i]

            for j in range(i + 1, m):
                if D[j, i] != 0:
                    q = D[j, i] // D[i, i]
                    D[j, :] -= q * D[i, :]
                    U[j, :] -= q * U[i, :]

        return U, D, V

    def _is_prime_power(self, n: int) -> bool:
        """Check if n is a prime power"""
        if n <= 1:
            return False

        for p in range(2, int(n**0.5) + 1):
            if n % p == 0:
                temp = n
                while temp % p == 0:
                    temp //= p
                return temp == 1

        return True  # n is prime

    def _enumerate_group_elements(self) -> List[str]:
        """Enumerate all elements of the group (for small groups)"""
        # This is a placeholder - in practice, we don't need to enumerate all elements
        return [f"g_{i}" for i in range(min(100, reduce(lambda x, y: x * y, self.orders, 1)))]

Quantum implementation using Classiq

In [11]:
@qfunc
def quantum_g_function(
    x: QArray[QBit],
    generators: List[QArray[QBit]],
    result: QArray[QBit]
) -> None:
    """
    Quantum implementation of the g function: Z_q^k → G
    Maps (x1, ..., xk) → a1^x1 * ... * ak^xk
    """
    # Initialize result to identity
    # This is a placeholder for actual group operations

    for i in range(len(generators)):
        # Controlled exponentiation: if x[i] is set, multiply by generators[i]
        # This would need to be implemented based on the specific group representation
        controlled_group_multiply(x[i], generators[i], result)


def controlled_group_multiply(control: QBit, generator: QArray[QBit], target: QArray[QBit]) -> None:
    """Placeholder for controlled group multiplication"""
    # This would implement actual group operations
    for j in range(len(target)):
        CX(control, target[j])

In [12]:
@qfunc
def quantum_algorithm_10(
    k: int,
    q: int,
    generator_size: int
) -> Tuple[QArray[QBit], QArray[QBit]]:
    """
    Quantum implementation of Algorithm 10
    """
    # Create registers
    x_reg = QArray("x", QBit, k * int(np.ceil(np.log2(q))))
    generators = [QArray(f"gen_{i}", QBit, generator_size) for i in range(k)]
    result_reg = QArray("result", QBit, generator_size)

    # Step 1: Create superposition over Z_q^k
    hadamard_transform(x_reg)

    # Step 2: Apply g function
    quantum_g_function(x_reg, generators, result_reg)

    # Step 3: Measure to find hidden subgroup
    # In practice, this would be followed by quantum Fourier transform
    # and classical post-processing

    return x_reg, result_reg

In [13]:
def example_algorithm_10():
    """
    Example usage of Algorithm 10 implementation
    """
    print("Algorithm 10 Implementation")
    print("=" * 50)

    # Example 1: Simple case with Z_4
    print("\nExample 1: Group with generator of order 4 (should give cyclic group Z_4)")
    try:
        alg = Algorithm10Implementation(
            generators=["a"],
            orders=[4],
            q=4
        )

        quotient_gens, final_gens = alg.decompose_group()

        print(f"Quotient generators: {[list(gen.coordinates) for gen in quotient_gens]}")
        print(f"Final generators: {final_gens}")
        print(f"Expected: Should decompose Z_4 into a single cyclic factor")

    except Exception as e:
        print(f"Error in Example 1: {e}")
        import traceback
        traceback.print_exc()

    # Example 2: Simpler case first - Z_2
    print("\nExample 2: simpler - generator of order 2")
    try:
        alg2 = Algorithm10Implementation(
            generators=["a"],
            orders=[2],
            q=2
        )

        quotient_gens2, final_gens2 = alg2.decompose_group()

        print(f"Quotient generators: {[list(gen.coordinates) for gen in quotient_gens2]}")
        print(f"Final generators: {final_gens2}")

    except Exception as e:
        print(f"Error in Example 2: {e}")
        import traceback
        traceback.print_exc()

    # Example 3: Two generators with different orders - but make q compatible
    print("\nExample 3: Two generators, orders [2, 2], q=2")
    try:
        alg3 = Algorithm10Implementation(
            generators=["a", "b"],
            orders=[2, 2],  # Both have order 2
            q=2  # q must be compatible
        )

        quotient_gens3, final_gens3 = alg3.decompose_group()

        print(f"Quotient generators: {[list(gen.coordinates) for gen in quotient_gens3]}")
        print(f"Final generators: {final_gens3}")
        print(f"Expected: Should give Z_2 ⊕ Z_2")

    except Exception as e:
        print(f"Error in Example 3: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    example_algorithm_10()

Algorithm 10 Implementation

Example 1: Group with generator of order 4 (should give cyclic group Z_4)
Starting Algorithm 10 with generators ['a'], q=4
Step 1: Finding hidden subgroup K...
Hidden subgroup K has 1 elements
Found 0 generators for hidden subgroup K
Step 2: Computing generators for Z_q^k / K...
Computing quotient generators using matrix method...
K is trivial, quotient is isomorphic to Z_q^k
Found 1 generators for quotient group
Step 3: Computing final group generators...
Quotient generators: [[1]]
Final generators: ['a']
Expected: Should decompose Z_4 into a single cyclic factor

Example 2: simpler - generator of order 2
Starting Algorithm 10 with generators ['a'], q=2
Step 1: Finding hidden subgroup K...
Hidden subgroup K has 1 elements
Found 0 generators for hidden subgroup K
Step 2: Computing generators for Z_q^k / K...
Computing quotient generators using matrix method...
K is trivial, quotient is isomorphic to Z_q^k
Found 1 generators for quotient group
Step 3: Comput